Using code from the following github repo: 
- https://github.com/codegiovanni/Analog_clock/blob/main/analog_clock.py
- Save pygame screen as video (option1): scrhttps://github.com/theRealProHacker/PyGameRecorder
- Save pygame screen as video (option2): http://github.com/ArjunSahlot/vidmaker
- Embed Matplotlib figures in Pygame screen: https://pypi.org/project/pygame-matplotlib/
- https://medium.com/data-storytelling-corner/how-python-voila-can-be-your-new-killer-visualization-superpower-a059c367d4e8

In [149]:
# Import packages
from collections import defaultdict
import datetime
from math import pi, cos, sin, atan2, radians
import pygame
# %gui qt

In [150]:
# Functions
def get_display_dimension():
    import tkinter
    root = tkinter.Tk()
    root.withdraw()
    w, h = root.winfo_screenwidth(), root.winfo_screenheight()
    return w, h

def get_screen_w_h(screen_ratio_w_h):
    display_w, display_h = get_display_dimension()
    return (display_w * screen_ratio_w_h[0], display_h * screen_ratio_w_h[1])

def get_screen_w_h_center(screen_ratio_w_h, make_plots, clock_w_ratio):
    screen_w_h = get_screen_w_h(screen_ratio_w_h)
    if make_plots:
        center = (clock_w_ratio * screen_w_h[0] / 2, screen_w_h[1] / 2)
    else:
        center = (screen_w_h [0]/ 2, screen_w_h[1] / 2)
    return screen_w_h, center
    
def display_value(value, font, size, position, color, bold=True, italic=False, return_rect=False):
    font = pygame.font.SysFont(font, size, bold, italic)
    text = font.render(value, True, color)
    text_rect = text.get_rect(center=(position))
    screen.blit(text, text_rect)
    if return_rect:
        return text_rect

def polar_to_cartesian(r, theta, xy_0=(0, 0)):
    x = r * sin(pi * theta / 180) + xy_0[0]
    y = -r * cos(pi * theta / 180) + xy_0[1]
    return x, y

def display_arrow(screen, color, start, r, theta, line_width, \
               arrow_size=50, arrow_angle=15, arrow_vee=False):
    """Draw an arrow on the given surface from start to end."""
    line_end = polar_to_cartesian(r-arrow_size/2, theta, xy_0=start)
    arrow_end = polar_to_cartesian(r, theta, xy_0=start)
    # Draw line
    pygame.draw.line(screen, color, start, line_end, line_width)

    # Calculate the angle of the line
    line_angle = atan2(arrow_end[1] - start[1], arrow_end[0] - start[0])

    # Calculate the points for the arrowhead
    arrow_p1 = (arrow_end[0] - arrow_size * cos(line_angle - radians(arrow_angle)),
                arrow_end[1] - arrow_size * sin(line_angle - radians(arrow_angle)))
    arrow_p2 = (arrow_end[0] - arrow_size * cos(line_angle + radians(arrow_angle)),
                arrow_end[1] - arrow_size * sin(line_angle + radians(arrow_angle)))

    # Draw the arrowhead as a filled polygon
    if arrow_vee:
        pygame.draw.polygon(screen, color, (arrow_end, arrow_p1, line_end, arrow_p2))
    else:
        pygame.draw.polygon(screen, color, (arrow_end, arrow_p1, arrow_p2))

def display_clock_base(screen, colors, center, clock_radius, number_font="Arial", number_size=80):
    screen.fill(colors['white'])
    pygame.draw.circle(screen, colors['black'], center, clock_radius-2, 10)
    pygame.draw.circle(screen, colors['black'], center, 12)
    # pygame.draw.rect(screen, colors['black'], [WIDTH / 2 - 260, HEIGHT / 2 - 30, 80, 60], 1)
    number_radius_adj = 80
    for number in range(1, 13):
        display_value(str(number), number_font, number_size, \
                   polar_to_cartesian(clock_radius-number_radius_adj, number*30, xy_0=center), colors['black'])

    for number in range(0, 360, 6):
        if number % 5:
            pygame.draw.line(screen, colors['black'], polar_to_cartesian(clock_radius - 15, number, xy_0=center),
                             polar_to_cartesian(clock_radius - 30, number, xy_0=center), 2)
        else:
            pygame.draw.line(screen, colors['black'], polar_to_cartesian(clock_radius - 15, number, xy_0=center),
                             polar_to_cartesian(clock_radius - 35, number, xy_0=center), 6)

def get_clock_time(clock_time, start_time, time_format, time_delta, display_cnt):
    if start_time == "now":
        return datetime.datetime.now()
    elif display_cnt == 1:
        try:
            return datetime.datetime.strptime(start_time, time_format)
        except Exception as err:
            print(f"Input Clock Start Time or Time Format Error: {err}")
            raise
    else:
        try:
            clock_time += datetime.timedelta(**time_delta)
            return clock_time
        except Exception as err:
            print(f"Input Clock Time Delta Error: {err}")
            raise
        
def display_analog_time(screen, clock_time, clock_center, display_seconds, clock_radius, \
                       clock_r_t, r_ratio_h_m_s, linewidth_h_m_s, colors):
    
    second = clock_time.second
    minute = clock_time.minute
    hour = clock_time.hour
    lw_h, lw_m, lw_s = linewidth_h_m_s

    # Hour
    h_radius = clock_radius*r_ratio_h_m_s[0]
    h_theta = (hour + minute / 60 + second / 3600) * (360 / 12)
    display_arrow(screen, colors['black'], clock_center, h_radius, h_theta, lw_h, arrow_size=lw_h*4)
    clock_r_t['h_radius'].append(h_radius)
    clock_r_t['h_theta'].append(h_theta)
    
    # Minute
    m_radius = clock_radius*r_ratio_h_m_s[1]
    m_theta = (minute + second / 60) * (360 / 60)
    display_arrow(screen, colors['black'], clock_center, m_radius, m_theta, lw_m, arrow_size=lw_m*4)
    clock_r_t['m_radius'].append(m_radius)
    clock_r_t['m_theta'].append(m_theta)
    
    # Second
    if display_seconds:
        s_radius = clock_radius*r_ratio_h_m_s[2]
        s_theta = second * (360 / 60)
        pygame.draw.line(screen, colors['black'], clock_center, \
                         polar_to_cartesian(s_radius, s_theta, xy_0=clock_center), lw_s)
        clock_r_t['s_radius'].append(s_radius)
        clock_r_t['s_theta'].append(s_theta)
    return clock_r_t

def get_minute_second_str(m_s):
    m_s_str = str(m_s)
    return m_s_str if len(m_s_str) == 2 else "0" + m_s_str
    
def get_clock_str(clock_time):
    pm_hours = {i for i in range(12, 24)}
    hour = clock_time.hour
    minute_str = get_minute_second_str(clock_time.minute)
    if hour == 0:
        clock_str = f"{hour+12}:{minute_str}"
        am_pm = "AM"
    elif hour in pm_hours:
        clock_str = f"{hour-12}:{minute_str}" if hour > 12 else f"{hour}:{minute_str}"
        am_pm = "PM"
    else:
        clock_str = f"{hour}:{minute_str}"
        am_pm = "AM"
    if display_seconds:
        second_str = get_minute_second_str(clock_time.second)
        clock_str = clock_str + f":{second_str}" + am_pm
    else:
        clock_str = clock_str + am_pm
    return clock_str
    
def display_digital_time(screen, screen_w_h, clock_w_ratio, clock_radius, \
                         clock_time, display_seconds, colors):
    number_size = 80
    digital_radius_buf = -40
    
    rect_loc_adj = (175, -50)
    rect_size = (350, 100)
    rect_lw = 2
    
    plots_center = (screen_w_h[0] * ((1 + clock_w_ratio) / 2), screen_w_h[1] / 2)
    digital_loc = polar_to_cartesian(clock_radius+digital_radius_buf, 0, xy_0=plots_center)
    
    clock_str = get_clock_str(clock_time)
    text_rect = display_value(clock_str, "Arial", number_size, digital_loc, colors['black'], \
                              bold=True, italic=False, return_rect=True)
    pygame.draw.rect(screen, colors['black'], [digital_loc[0]-rect_loc_adj[0], digital_loc[1]--rect_loc_adj[1], *rect_size], rect_lw)
    
def display_re_im_axes(screen, colors, center, clock_radius, axes_radius_buf, axes_linewidth, arrow_vee=True):
    # Draw Real and Imaginary Axes
    display_arrow(screen, colors['red'], center, clock_radius+axes_radius_buf, 0, axes_linewidth, \
               arrow_size=axes_linewidth*6, arrow_angle=30, arrow_vee=arrow_vee)
    display_arrow(screen, colors['red'], center, clock_radius+axes_radius_buf, 180, axes_linewidth, \
               arrow_size=axes_linewidth*6, arrow_angle=30, arrow_vee=arrow_vee)     
    display_arrow(screen, colors['blue'], center, clock_radius+axes_radius_buf, 90, axes_linewidth, \
               arrow_size=axes_linewidth*6, arrow_angle=30, arrow_vee=arrow_vee)
    display_arrow(screen, colors['blue'], center, clock_radius+axes_radius_buf, 270, axes_linewidth, \
               arrow_size=axes_linewidth*6, arrow_angle=30, arrow_vee=arrow_vee)  
    # Draw Real (Re) and Imaginary (Im) labels
    display_value('+Im', "latinmodernmath", 30, polar_to_cartesian(clock_radius+axes_radius_buf+5, 4, xy_0=center), \
               colors['red'], bold=False)
    display_value('-Im', "latinmodernmath", 30, polar_to_cartesian(clock_radius+axes_radius_buf+5, 184, xy_0=center), \
               colors['red'], bold=False)
    display_value('+Re', "latinmodernmath", 30, polar_to_cartesian(clock_radius+axes_radius_buf+10, 87, xy_0=center), \
               colors['blue'], bold=False)
    display_value('-Re', "latinmodernmath", 30, polar_to_cartesian(clock_radius+axes_radius_buf+10, 267, xy_0=center), \
               colors['blue'], bold=False)
    # Draw tick marks showing scale
    # Imaginary 
    im_scale_loc0 = polar_to_cartesian(clock_radius, 0, xy_0=center)
    im_scale_loc1 = (im_scale_loc0[0]-12, im_scale_loc0[1])
    pygame.draw.line(screen, colors['red'], im_scale_loc0, im_scale_loc1, 4)
    display_value('1', "latinmodernmath", 30, polar_to_cartesian(clock_radius+9,356, xy_0=center), colors['red'])
    # Real
    re_scale_loc0 = polar_to_cartesian(clock_radius, 90, xy_0=center)
    re_scale_loc1 = (re_scale_loc0[0], re_scale_loc0[1]+12)
    pygame.draw.line(screen, colors['blue'], re_scale_loc0, re_scale_loc1, 4)
    display_value('1', "latinmodernmath", 30, polar_to_cartesian(clock_radius+9, 94, xy_0=center), colors['blue'])
    
def print_available_math_fonts():
    print(*[fnt for fnt in pygame.font.get_fonts() if 'math' in fnt], sep='\n')

In [151]:
# Set parameters

# start_time = "now"
start_time = "00:00:00"
time_format = "%H:%M:%S"
time_delta = {'hours': 0, 'minutes':59, 'seconds': 0}
display_seconds = False
fps = 1

make_plots = True
# screen_ratio_w_h = (.5, .9)
screen_ratio_w_h = (.95, .9)
clock_w_ratio = .6
clock_radius = 400
r_ratio_h_m_s = (.61, .74, .9)
linewidth_h_m_s = (16, 12, 6)

colors = {'white': (255, 255, 255), \
          'black': (0, 0, 0), \
          'red': (255, 0, 0), \
          'blue': (0, 0, 255)}

axes_radius_buf = 35
axes_linewidth = 4

In [155]:
# Run main function
screen_w_h, clock_center = get_screen_w_h_center(screen_ratio_w_h, make_plots, clock_w_ratio)

pygame.init()
screen = pygame.display.set_mode(screen_w_h)
pygame.display.set_caption("Complex Number Clock")
clock = pygame.time.Clock()

def main():
    run = True
    display_cnt = 0
    clock_time = None
    clock_radius_theta = defaultdict(list)
    while run:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
        display_cnt += 1      
        
        display_clock_base(screen, colors, clock_center, clock_radius)
        
        display_re_im_axes(screen, colors, clock_center, clock_radius, axes_radius_buf, axes_linewidth)
        
        clock_time = get_clock_time(clock_time, start_time, time_format, time_delta, display_cnt)

        clock_radius_theta = display_analog_time(screen, clock_time, clock_center, display_seconds, \
                                    clock_radius, clock_radius_theta, r_ratio_h_m_s, linewidth_h_m_s, colors)

        if make_plots:
            display_digital_time(screen, screen_w_h, clock_w_ratio, clock_radius, \
                                 clock_time, display_seconds, colors)
        
        pygame.display.update()

        clock.tick(fps)

    pygame.quit()
    print(clock_radius_theta)

main()

defaultdict(<class 'list'>, {'h_radius': [244.0, 244.0, 244.0, 244.0, 244.0, 244.0, 244.0], 'h_theta': [360.0, 389.5, 419.0, 448.5, 478.0, 507.50000000000006, 537.0], 'm_radius': [296.0, 296.0, 296.0, 296.0, 296.0, 296.0, 296.0], 'm_theta': [0.0, 354.0, 348.0, 342.0, 336.0, 330.0, 324.0]})


In [69]:
# Quit if there is an error to close screen
pygame.quit()